# Expectations

In [52]:
import great_expectations as ge
from great_expectations.checkpoint import SimpleCheckpoint
import pandas as pd
import logging
import yaml
from typing import Dict, Tuple, Any
import os

logger = logging.getLogger(__name__)

In [22]:
df = pd.read_csv("../data/01_raw/house-pricing.csv")
df_model_input = pd.read_csv("../data/05_model_input/X_train_transformed.csv")

In [23]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [24]:
gdf = ge.from_pandas(df)
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

Check column amount

In [5]:
column_number = len(gdf.columns)
gdf.expect_table_column_count_to_equal(column_number)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "observed_value": 81
  },
  "success": true
}

Check if column exist

In [6]:
column_list = gdf.columns.tolist()

In [7]:
column_list_yaml = yaml.dump(column_list)
print(column_list_yaml)

- Id
- MSSubClass
- MSZoning
- LotFrontage
- LotArea
- Street
- Alley
- LotShape
- LandContour
- Utilities
- LotConfig
- LandSlope
- Neighborhood
- Condition1
- Condition2
- BldgType
- HouseStyle
- OverallQual
- OverallCond
- YearBuilt
- YearRemodAdd
- RoofStyle
- RoofMatl
- Exterior1st
- Exterior2nd
- MasVnrType
- MasVnrArea
- ExterQual
- ExterCond
- Foundation
- BsmtQual
- BsmtCond
- BsmtExposure
- BsmtFinType1
- BsmtFinSF1
- BsmtFinType2
- BsmtFinSF2
- BsmtUnfSF
- TotalBsmtSF
- Heating
- HeatingQC
- CentralAir
- Electrical
- 1stFlrSF
- 2ndFlrSF
- LowQualFinSF
- GrLivArea
- BsmtFullBath
- BsmtHalfBath
- FullBath
- HalfBath
- BedroomAbvGr
- KitchenAbvGr
- KitchenQual
- TotRmsAbvGrd
- Functional
- Fireplaces
- FireplaceQu
- GarageType
- GarageYrBlt
- GarageFinish
- GarageCars
- GarageArea
- GarageQual
- GarageCond
- PavedDrive
- WoodDeckSF
- OpenPorchSF
- EnclosedPorch
- 3SsnPorch
- ScreenPorch
- PoolArea
- PoolQC
- Fence
- MiscFeature
- MiscVal
- MoSold
- YrSold
- SaleType
- SaleCondi

In [8]:
def check_if_column_exist(gdf, column_list):
    for column in column_list:
        gdf.expect_column_to_exist(column)

In [9]:
check_if_column_exist(gdf, column_list)

Check dtypes

In [10]:
def check_dtype(gdf, columns, dtype):
    if dtype == 'numeric':
        for column in columns:
            gdf.expect_column_values_to_be_in_type_list(column, ['int64', 'float64'])
    else:
        for column in columns:
            gdf.expect_column_values_to_be_in_type_list(column, ["str"])

In [11]:
check_dtype(gdf, num_cols, dtype='numeric')
check_dtype(gdf, cat_cols, dtype='object')

Check unique values of categorical columns

In [12]:
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols_dict = {}
for column in cat_cols:
    cat_cols_dict[column] = df[column].unique().tolist()

In [13]:
cat_cols_dict

{'MSZoning': ['RL', 'RM', 'C (all)', 'FV', 'RH'],
 'Street': ['Pave', 'Grvl'],
 'Alley': [nan, 'Grvl', 'Pave'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'LandContour': ['Lvl', 'Bnk', 'Low', 'HLS'],
 'Utilities': ['AllPub', 'NoSeWa'],
 'LotConfig': ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'Neighborhood': ['CollgCr',
  'Veenker',
  'Crawfor',
  'NoRidge',
  'Mitchel',
  'Somerst',
  'NWAmes',
  'OldTown',
  'BrkSide',
  'Sawyer',
  'NridgHt',
  'NAmes',
  'SawyerW',
  'IDOTRR',
  'MeadowV',
  'Edwards',
  'Timber',
  'Gilbert',
  'StoneBr',
  'ClearCr',
  'NPkVill',
  'Blmngtn',
  'BrDale',
  'SWISU',
  'Blueste'],
 'Condition1': ['Norm',
  'Feedr',
  'PosN',
  'Artery',
  'RRAe',
  'RRNn',
  'RRAn',
  'PosA',
  'RRNe'],
 'Condition2': ['Norm',
  'Artery',
  'RRNn',
  'Feedr',
  'PosN',
  'PosA',
  'RRAn',
  'RRAe'],
 'BldgType': ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],
 'HouseStyle': ['2Story',
  '1Story',
  '1.5Fin',
  '1.5Unf',
  

In [14]:
import yaml
result = yaml.dump(cat_cols_dict)
print(result)

Alley:
- .nan
- Grvl
- Pave
BldgType:
- 1Fam
- 2fmCon
- Duplex
- TwnhsE
- Twnhs
BsmtCond:
- TA
- Gd
- .nan
- Fa
- Po
BsmtExposure:
- 'No'
- Gd
- Mn
- Av
- .nan
BsmtFinType1:
- GLQ
- ALQ
- Unf
- Rec
- BLQ
- .nan
- LwQ
BsmtFinType2:
- Unf
- BLQ
- .nan
- ALQ
- Rec
- LwQ
- GLQ
BsmtQual:
- Gd
- TA
- Ex
- .nan
- Fa
CentralAir:
- Y
- N
Condition1:
- Norm
- Feedr
- PosN
- Artery
- RRAe
- RRNn
- RRAn
- PosA
- RRNe
Condition2:
- Norm
- Artery
- RRNn
- Feedr
- PosN
- PosA
- RRAn
- RRAe
Electrical:
- SBrkr
- FuseF
- FuseA
- FuseP
- Mix
- .nan
ExterCond:
- TA
- Gd
- Fa
- Po
- Ex
ExterQual:
- Gd
- TA
- Ex
- Fa
Exterior1st:
- VinylSd
- MetalSd
- Wd Sdng
- HdBoard
- BrkFace
- WdShing
- CemntBd
- Plywood
- AsbShng
- Stucco
- BrkComm
- AsphShn
- Stone
- ImStucc
- CBlock
Exterior2nd:
- VinylSd
- MetalSd
- Wd Shng
- HdBoard
- Plywood
- Wd Sdng
- CmentBd
- BrkFace
- Stucco
- AsbShng
- Brk Cmn
- ImStucc
- AsphShn
- Stone
- Other
- CBlock
Fence:
- .nan
- MnPrv
- GdWo
- GdPrv
- MnWw
FireplaceQu:
- .nan
- TA
-

In [15]:
def check_categorical_unique_values(gdf, dict_cat_cols):
    for column in dict_cat_cols.keys():
        gdf.expect_column_values_to_be_in_set(column, dict_cat_cols[column])

In [16]:
check_categorical_unique_values(gdf, cat_cols_dict)

Check SalePrice deviation

In [17]:
# def calculate_median_deviation(gdf, column):
#     df = gdf.copy()
#     median = df[column].median()
#     absolute_deviations = df[column].apply(lambda x: abs(x - median))
#     mad = absolute_deviations.median()
#     return mad

In [18]:
median_sales_price = df['SalePrice'].median()
median_threshold = 0.1
gdf.expect_column_median_to_be_between("SalePrice", median_sales_price*(1-median_threshold), median_sales_price*(1+median_threshold))

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "observed_value": 163000.0,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

Check columns not be null

In [19]:
def check_nulls(gdf, columns):

    for column in columns:
        gdf.expect_column_values_to_not_be_null(column)

In [20]:
check_nulls(gdf, gdf.columns)

Check if ID is unique

In [21]:
# check if id is unique
gdf.expect_column_values_to_be_unique("Id")

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [26]:
import datetime

current_year = datetime.datetime.now().year

In [27]:
gdf.expect_column_values_to_be_between("YearBuilt", 1800, current_year)

{
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {}
}

In [19]:
gdf.expect_column_max_to_be_between("YrSold", 1950, current_year)


NameError: name 'gdf' is not defined

In [25]:
gdf[num_cols]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [28]:
validation = gdf.validate()

In [45]:
validation.to_json_dict()

{'meta': {'great_expectations_version': '0.17.1',
  'expectation_suite_name': 'default',
  'run_id': {'run_name': None, 'run_time': '2023-06-26T13:01:30.322681+01:00'},
  'batch_kwargs': {'ge_batch_id': '293db3dc-1419-11ee-9f06-84144d01abb6'},
  'batch_markers': {},
  'batch_parameters': {},
  'validation_time': '20230626T120130.322681Z',
  'expectation_suite_meta': {'great_expectations_version': '0.17.1'}},
 'evaluation_parameters': {},
 'statistics': {'evaluated_expectations': 1,
  'successful_expectations': 1,
  'unsuccessful_expectations': 0,
  'success_percent': 100.0},
 'success': True,
 'results': [{'result': {'element_count': 1460,
    'missing_count': 0,
    'missing_percent': 0.0,
    'unexpected_count': 0,
    'unexpected_percent': 0.0,
    'unexpected_percent_total': 0.0,
    'unexpected_percent_nonmissing': 0.0,
    'partial_unexpected_list': []},
   'expectation_config': {'expectation_type': 'expect_column_values_to_be_between',
    'kwargs': {'column': 'YearBuilt',
     

In [32]:
folder_path = '../data/08_reporting/Expectations_reporting'
os.makedirs(folder_path, exist_ok=True)

validation_results = gdf.validate()
print(folder_path + "/feature_engineered_data_validation_results.json")

../data/08_reporting/Expectations_reportingfeature_engineered_data_validation_results.json


In [47]:
import json
validation_results = gdf.validate()
file_path_validation_results = os.path.join(folder_path, "feature_engineered_data_validation_results.json")
with open(file_path_validation_results, 'w') as json_file:
    json.dump(validation_results.to_json_dict(), json_file)

In [41]:
validation_results.to_json_dict("../data/08_reporting/Expectations_reporting/feature_engineered_data_validation_results.json")

TypeError: to_json_dict() takes 1 positional argument but 2 were given

In [28]:
logger = logging.getLogger(__name__)

In [ ]:
failed_expectations = [result for result in validation["results"] if not result["success"]]
    
logger.info(
        f"Total Expectations: {len(validation['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"Failed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )

In [49]:
import json
import os
# Create the data/08_reporting folder if it doesn't exist
folder_path = '../data/08_reporting'
os.makedirs(folder_path, exist_ok=True)

# Define the file path
file_path = os.path.join(folder_path, 'collect_errors.json')

# Save the list as a JSON file
with open(file_path, 'w') as json_file:
    json.dump(collect_errors, json_file)

print(f"The JSON file has been saved at {file_path}.")

The JSON file has been saved at ../data/08_reporting\collect_errors.json.


Node function

In [ ]:
def check_data_cleaning(df: pd.DataFrame, parameters : Dict[str, Any]) -> Tuple[pd.DataFrame, Dict]:
    """
    Check expectations for the cleaned dataset.
    - Check if the number of columns.
    - Check if the columns exist.
    - Check if the columns are of the correct type.
    - Check if the categorical columns have the correct unique values.
    - Check if the numeric columns are within the correct range.
    - Check if the columns have null values.
    - Check if the ID has only unique values.
    - Check if the median of the SalePrice is within the threshold.
    - Check if the YearBuilt and YrSold are within the correct range.

    Afterwards save the validation results and raise an exception and save the errors, if any of the expectations fail.

    Args:
    --
        df (pd.DataFrame): Dataframe to check for nulls.
        parameters (Dict): Parameters from the configuration file.

    Returns:
    --
        df (pd.DataFrame): Input dataframe.
    """

    # Creates a folder to save the expectations results.
    folder_path = '../data/08_reporting/Expectations_reporting'
    os.makedirs(folder_path, exist_ok=True)

    current_year = datetime.date.today().year + 1
    num_cols = df.select_dtypes(include=['number']).columns.tolist()
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    num_columns = parameters["num_columns"]
    column_list = parameters["column_list"]
    cat_unique_values = parameters["categorical_unique_values"]
    median_sales_price = parameters["median_sales_price"]
    median_threshold = parameters["median_threshold"]

    ranges = parameters["num_quality_ranges"]
    gdf = ge.from_pandas(df)

    gdf.expect_table_column_count_to_equal(num_columns)
    check_if_column_exist(gdf, column_list)
    check_dtype(gdf, num_cols, dtype='numeric')
    check_dtype(gdf, cat_cols, dtype='object')
    check_categorical_unique_values(gdf, cat_unique_values)
    
    gdf.expect_column_median_to_be_between("SalePrice", 
                                           median_sales_price*(1-median_threshold), 
                                           median_sales_price*(1+median_threshold))
    check_nulls(gdf, gdf.columns)
    gdf.expect_column_values_to_be_unique("Id")
    gdf.expect_column_max_to_be_between("YearBuilt", 1800, current_year)
    gdf.expect_column_max_to_be_between("YrSold", 1950, current_year)
    
    # Create the validation results and save them in a json file.
    validation_results = gdf.validate()
    file_path_validation_results = os.path.join(folder_path, "Cleaned_data_validation_results.json")
    validation_results.to_json_dict(file_path_validation_results)

    failed_expectations = [result for result in validation_results["results"] if not result["success"]]
    
    logger.info(
        f"Total Expectations: {len(validation_results['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
    # Collects the errors in a list and saves them in a json file.
    # Afterwards raises an exception with the errors.
    if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"  Failed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
            
            # Saves the collected errors in a json file.
            file_path = os.path.join(folder_path, 'Cleaned_data_errors.json')
            with open(file_path, 'w') as json_file:
                json.dump(collect_errors, json_file)
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )
      
    return df

In [56]:
def check_data_feature_engineering(df: pd.DataFrame, parameters : Dict[str, Any]) -> Tuple[pd.DataFrame, Dict]:
    """
    Check expectations for the feature engineered dataset.
    - Check if the numerical features are in the expected range.
    - Check if the onehotencoded categorical features have values 0 or 1.

    Afterwards save the validation results and raise an exception and save the errors, if any of the expectations fail.

    Args:
    --
        df (pd.DataFrame): Dataframe to check for nulls.
        parameters (Dict): Parameters from the configuration file.

    Returns:
    --
        df (pd.DataFrame): Input dataframe.
    """

    df = df.copy().drop(columns=['y_pred'])
    
    folder_path = '../data/08_reporting/Expectations_reporting'
    os.makedirs(folder_path, exist_ok=True)
    
    num_cols = df.select_dtypes(include=['number']).columns
    cat_cols = df.select_dtypes(include=['object']).columns

    ohencoded_values = [0,1]
    ranges = {'min': -5, 'max': 5}

    gdf = ge.from_pandas(df)
    for column in num_cols:
        gdf.expect_column_values_to_be_between(column,ranges['min'],ranges['max'])

    for column in cat_cols:
        gdf.expect_column_values_to_be_in_set(column, ohencoded_values)

    # Create the validation results and save them in a json file.
    validation_results = gdf.validate()
    file_path_validation_results = os.path.join(folder_path, "feature_engineered_data_validation_results.json")
    with open(file_path_validation_results, 'w') as json_file:
        json.dump(validation_results.to_json_dict(), json_file)

    failed_expectations = [result for result in validation_results["results"] if not result["success"]]
    
    logger.info(
        f"Total Expectations: {len(validation_results['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
    # Collects the errors in a list and saves them in a json file.
    # Afterwards raises an exception with the errors.
    if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"  Failed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
            
            # Saves the collected errors in a json file.
            file_path = os.path.join(folder_path, 'feature_engineered_data_errors.json')
            with open(file_path, 'w') as json_file:
                json.dump(collect_errors, json_file)
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )
   
    return df

In [57]:
import pandas as pd
test_df = pd.read_csv('../data/07_model_output/df_with_predict.csv')

In [58]:
check_data_feature_engineering(test_df, None)

Exception: Data Quality Validation Failed: ["  Failed Expectation 1:  Expectation Type: expect_column_values_to_be_between  Column: numerical__numerical__LotFrontage  Details: {'element_count': 292, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 1, 'unexpected_percent': 0.3424657534246575, 'unexpected_percent_total': 0.3424657534246575, 'unexpected_percent_nonmissing': 0.3424657534246575, 'partial_unexpected_list': [5.455586916818861]}", "  Failed Expectation 2:  Expectation Type: expect_column_values_to_be_between  Column: numerical__numerical__LotArea  Details: {'element_count': 292, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 4, 'unexpected_percent': 1.36986301369863, 'unexpected_percent_total': 1.36986301369863, 'unexpected_percent_nonmissing': 1.36986301369863, 'partial_unexpected_list': [6.090815148015318, 5.6088991550665614, 14.871071667375844, 7.173788827426904]}", "  Failed Expectation 3:  Expectation Type: expect_column_values_to_be_betwee